In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
"""
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
"""
# Any results you write to the current directory are saved as output.

"\nimport os\nfor dirname, _, filenames in os.walk('/kaggle/input'):\n    for filename in filenames:\n        print(os.path.join(dirname, filename))\n"

# Data Exploration

In [2]:
train_df = pd.read_csv('/kaggle/input/champs-scalar-coupling/train.csv')
train_df.head(10)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074
5,5,dsgdb9nsd_000001,2,3,2JHH,-11.2541
6,6,dsgdb9nsd_000001,2,4,2JHH,-11.2548
7,7,dsgdb9nsd_000001,3,0,1JHC,84.8093
8,8,dsgdb9nsd_000001,3,4,2JHH,-11.2543
9,9,dsgdb9nsd_000001,4,0,1JHC,84.8095


In [3]:
train_df['type'].unique()

array(['1JHC', '2JHH', '1JHN', '2JHN', '2JHC', '3JHH', '3JHC', '3JHN'],
      dtype=object)

In [4]:
# Mulliken Charges
pd.read_csv('/kaggle/input/champs-scalar-coupling/mulliken_charges.csv').head()

,molecule_name,atom_index,mulliken_charge
0,dsgdb9nsd_000001,0,-0.535689
1,dsgdb9nsd_000001,1,0.133921
2,dsgdb9nsd_000001,2,0.133922
3,dsgdb9nsd_000001,3,0.133923
4,dsgdb9nsd_000001,4,0.133923


In [5]:
# Structures
pd.read_csv('/kaggle/input/champs-scalar-coupling/structures.csv').head()

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397


In [6]:
# Potential Energy
pd.read_csv('/kaggle/input/champs-scalar-coupling/potential_energy.csv').head()

,molecule_name,potential_energy
0,dsgdb9nsd_000001,-40.523680
1,dsgdb9nsd_000002,-56.560246
2,dsgdb9nsd_000003,-76.426077
3,dsgdb9nsd_000004,-77.335268
4,dsgdb9nsd_000005,-93.428488


In [7]:
# Scalar Coupling Contributions
pd.read_csv('/kaggle/input/champs-scalar-coupling/scalar_coupling_contributions.csv').head()

,molecule_name,atom_index_0,atom_index_1,type,fc,sd,pso,dso
0,dsgdb9nsd_000001,1,0,1JHC,83.0224,0.254579,1.25862,0.272010
1,dsgdb9nsd_000001,1,2,2JHH,-11.0347,0.352978,2.85839,-3.433600
2,dsgdb9nsd_000001,1,3,2JHH,-11.0325,0.352944,2.85852,-3.433870
3,dsgdb9nsd_000001,1,4,2JHH,-11.0319,0.352934,2.85855,-3.433930
4,dsgdb9nsd_000001,2,0,1JHC,83.0222,0.254585,1.25861,0.272013


In [8]:
# Magnetic Shielding Tensors
pd.read_csv('/kaggle/input/champs-scalar-coupling/magnetic_shielding_tensors.csv').head()

,molecule_name,atom_index,XX,YX,ZX,XY,YY,ZY,XZ,YZ,ZZ
0,dsgdb9nsd_000001,0,195.3147,0.0000,-0.0001,0.0000,195.3171,0.0007,-0.0001,0.0007,195.3169
1,dsgdb9nsd_000001,1,31.3410,-1.2317,4.0544,-1.2317,28.9546,-1.7173,4.0546,-1.7173,34.0861
2,dsgdb9nsd_000001,2,31.5814,1.2173,-4.1474,1.2173,28.9036,-1.6036,-4.1476,-1.6036,33.8967
3,dsgdb9nsd_000001,3,31.5172,4.1086,1.2723,4.1088,33.9068,1.6950,1.2724,1.6951,28.9579
4,dsgdb9nsd_000001,4,31.4029,-4.0942,-1.1793,-4.0944,34.0776,1.6259,-1.1795,1.6260,28.9013


In [9]:
# Dipole Moments
pd.read_csv('/kaggle/input/champs-scalar-coupling/dipole_moments.csv').head()

,molecule_name,X,Y,Z
0,dsgdb9nsd_000001,0.0000,0.0,0.0000
1,dsgdb9nsd_000002,-0.0002,0.0,1.6256
2,dsgdb9nsd_000003,0.0000,0.0,-1.8511
3,dsgdb9nsd_000004,0.0000,0.0,0.0000
4,dsgdb9nsd_000005,0.0000,0.0,-2.8937


In [10]:
# Sample Submission
pd.read_csv('/kaggle/input/champs-scalar-coupling/sample_submission.csv').head(10)

,id,scalar_coupling_constant
0,4659076,0
1,4659077,0
2,4659078,0
3,4659079,0
4,4659080,0
5,4659081,0
6,4659082,0
7,4659083,0
8,4659084,0
9,4659085,0


# Data Wrangling

In [11]:
def dataframe_creator(left_df,right_path,left_key,right_key,rename_dict,drop_list):
    """
    This Function will create the new dataframe by merging two dataframe on the basis of their keys
    
    Prameters----------------------------------------------------------------------- 
    
    left_df: Left Dataframe instance
    right_path: Path of the right dataframe
    left_key: A tuple which contains two list respective to atom index
    right_key: A list of right dataframe's key
    rename_dict: Dictionary consists of original column_name : new column_name pair
    drop_list: This list consists of column name which need to be dropped
    
    --------------------------------------------------------------------------------
    
    ----------------------------------
    return a new modified dataframe
    ----------------------------------
    """
    train_df = train_df = pd.merge(left_df,pd.read_csv(right_path),
                     left_on= left_key[0],right_on= right_key)
    train_df = pd.merge(train_df,pd.read_csv(right_path),
                     left_on= left_key[1],right_on= right_key)

    train_df.rename(columns= rename_dict,inplace=True)
    train_df.drop(columns= drop_list,inplace=True)
    
    return train_df

In [12]:
# Data Frame Creation

# Step-1: Joining the Train set and Mulliken Charges on the basis of keys:- Train_set[Molecule_id,atom_index0] and Mulliken_Charges[Molecule_id, atom_index]
train_df = dataframe_creator(left_df= train_df,right_path= '/kaggle/input/champs-scalar-coupling/mulliken_charges.csv',
                            left_key = (['molecule_name','atom_index_0'],['molecule_name','atom_index_1']), right_key = ['molecule_name','atom_index'],
                            rename_dict = {'mulliken_charge_x':'mulliken_charge_0', 'mulliken_charge_y':'mulliken_charge_1'}, 
                            drop_list= ['atom_index_x','atom_index_y'])

# Step-2: Joining the Train set and Structures on the basis of keys:- Train_set[Molecule_id,atom_index0] and Structures[Molecule_id, atom_index]
train_df = dataframe_creator(left_df= train_df,right_path= '/kaggle/input/champs-scalar-coupling/structures.csv',
                             left_key = (['molecule_name','atom_index_0'],['molecule_name','atom_index_1']), right_key = ['molecule_name','atom_index'],
                             rename_dict = {'atom_x':'atom_0', 'atom_y':'atom_1', 'x_x':'x_0', 'x_y':'x_1', 'y_x':'y_0', 'y_y':'y_1', 'z_x':'z_0',
                                            'z_y':'z_1'}, 
                             drop_list= ['atom_index_x','atom_index_y'])

# Step-3:  Joining the Train set and Potential Energy on the basis of Molecule_name key:
train_df = pd.merge(train_df,pd.read_csv('/kaggle/input/champs-scalar-coupling/potential_energy.csv'),on='molecule_name')

# Step-4: Joining the Train set and Scalar Coupling Contributions on the basis of keys:- [molecule_name, atom_index_0, atom_index_1]
train_df= pd.merge(train_df, pd.read_csv('/kaggle/input/champs-scalar-coupling/scalar_coupling_contributions.csv'),
                   on=['molecule_name','atom_index_0','atom_index_1'])
train_df.drop('type_y',axis=1,inplace=True)
train_df.rename(columns={'type_x':'type'},inplace=True)

# Step-5: Joining Train Set and Magnetic Shielding Tensors on the basis of keys:- [molecule_name,atom_index_0/atom_index_1]
col1 = pd.read_csv('/kaggle/input/champs-scalar-coupling/magnetic_shielding_tensors.csv').columns[2:]
rename_dict = {}
for i in col1:
    for ui,j in zip([i+'_x', i+'_y'],['0','1']):
        rename_dict[ui] =i+'mag_'+j

rename_dict

train_df = dataframe_creator(left_df= train_df,right_path= '/kaggle/input/champs-scalar-coupling/magnetic_shielding_tensors.csv',
                             left_key = (['molecule_name','atom_index_0'],['molecule_name','atom_index_1']), right_key = ['molecule_name','atom_index'],
                             rename_dict = rename_dict, 
                             drop_list= ['atom_index_x','atom_index_y'])

train_df.head(10)


,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,mulliken_charge_0,mulliken_charge_1,atom_0,x_0,...,ZZmag_0,XXmag_1,YXmag_1,ZXmag_1,XYmag_1,YYmag_1,ZYmag_1,XZmag_1,YZmag_1,ZZmag_1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,0.133921,-0.535689,H,0.002150,...,34.0861,195.3147,0.0000,-0.0001,0.0000,195.3171,0.0007,-0.0001,0.0007,195.3169
1,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,0.133922,-0.535689,H,1.011731,...,33.8967,195.3147,0.0000,-0.0001,0.0000,195.3171,0.0007,-0.0001,0.0007,195.3169
2,7,dsgdb9nsd_000001,3,0,1JHC,84.8093,0.133923,-0.535689,H,-0.540815,...,28.9579,195.3147,0.0000,-0.0001,0.0000,195.3171,0.0007,-0.0001,0.0007,195.3169
3,9,dsgdb9nsd_000001,4,0,1JHC,84.8095,0.133923,-0.535689,H,-0.523814,...,28.9013,195.3147,0.0000,-0.0001,0.0000,195.3171,0.0007,-0.0001,0.0007,195.3169
4,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,0.133921,0.133922,H,0.002150,...,34.0861,31.5814,1.2173,-4.1474,1.2173,28.9036,-1.6036,-4.1476,-1.6036,33.8967
5,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,0.133921,0.133923,H,0.002150,...,34.0861,31.5172,4.1086,1.2723,4.1088,33.9068,1.6950,1.2724,1.6951,28.9579
6,5,dsgdb9nsd_000001,2,3,2JHH,-11.2541,0.133922,0.133923,H,1.011731,...,33.8967,31.5172,4.1086,1.2723,4.1088,33.9068,1.6950,1.2724,1.6951,28.9579
7,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,0.133921,0.133923,H,0.002150,...,34.0861,31.4029,-4.0942,-1.1793,-4.0944,34.0776,1.6259,-1.1795,1.6260,28.9013
8,6,dsgdb9nsd_000001,2,4,2JHH,-11.2548,0.133922,0.133923,H,1.011731,...,33.8967,31.4029,-4.0942,-1.1793,-4.0944,34.0776,1.6259,-1.1795,1.6260,28.9013
9,8,dsgdb9nsd_000001,3,4,2JHH,-11.2543,0.133923,0.133923,H,-0.540815,...,28.9579,31.4029,-4.0942,-1.1793,-4.0944,34.0776,1.6259,-1.1795,1.6260,28.9013


In [13]:
train_df.columns

Index(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'type',
       'scalar_coupling_constant', 'mulliken_charge_0', 'mulliken_charge_1',
       'atom_0', 'x_0', 'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1',
       'potential_energy', 'fc', 'sd', 'pso', 'dso', 'XXmag_0', 'YXmag_0',
       'ZXmag_0', 'XYmag_0', 'YYmag_0', 'ZYmag_0', 'XZmag_0', 'YZmag_0',
       'ZZmag_0', 'XXmag_1', 'YXmag_1', 'ZXmag_1', 'XYmag_1', 'YYmag_1',
       'ZYmag_1', 'XZmag_1', 'YZmag_1', 'ZZmag_1'],
      dtype='object')

In [14]:
train_df.shape

(4659076, 39)

In [15]:
train_df.columns

Index(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'type',
       'scalar_coupling_constant', 'mulliken_charge_0', 'mulliken_charge_1',
       'atom_0', 'x_0', 'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1',
       'potential_energy', 'fc', 'sd', 'pso', 'dso', 'XXmag_0', 'YXmag_0',
       'ZXmag_0', 'XYmag_0', 'YYmag_0', 'ZYmag_0', 'XZmag_0', 'YZmag_0',
       'ZZmag_0', 'XXmag_1', 'YXmag_1', 'ZXmag_1', 'XYmag_1', 'YYmag_1',
       'ZYmag_1', 'XZmag_1', 'YZmag_1', 'ZZmag_1'],
      dtype='object')

# Categorical Encodings

In [16]:
train_df['atom_0'].unique()

array(['H'], dtype=object)

In [17]:
train_df['atom_1'].unique()

array(['C', 'H', 'N'], dtype=object)

In [18]:
train_df['type'].unique()

array(['1JHC', '2JHH', '1JHN', '2JHN', '2JHC', '3JHH', '3JHC', '3JHN'],
      dtype=object)

In [19]:
# Extracting type Numeric
type_suffix = train_df['type'].apply(lambda x: x[0])
type_suffix

0          1
1          1
2          1
3          1
4          2
          ..
4659071    3
4659072    3
4659073    3
4659074    3
4659075    3
Name: type, Length: 4659076, dtype: object

In [20]:
type_suffix = pd.get_dummies(type_suffix)
type_suffix.drop('3',axis=1,inplace=True)
type_suffix.head(10)

,1,2
0,1,0
1,1,0
2,1,0
3,1,0
4,0,1
5,0,1
6,0,1
7,0,1
8,0,1
9,0,1


In [21]:

atom_dict = {'H':0,'N':1,'C':2}
atom_0_dummies = train_df['atom_0'].apply(lambda x: atom_dict[x])
atom_0_dummies.head(10)

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: atom_0, dtype: int64

In [22]:
atom_1_dummies = train_df['atom_1'].apply(lambda x: atom_dict[x])
atom_1_dummies = pd.get_dummies(atom_1_dummies,prefix='atom1')
atom_1_dummies.drop('atom1_2',axis=1,inplace=True)
atom_1_dummies.head(10)

,atom1_0,atom1_1
0,0,0
1,0,0
2,0,0
3,0,0
4,1,0
5,1,0
6,1,0
7,1,0
8,1,0
9,1,0


In [23]:
# Step1: Deleting the type column from the dataset and adding the type_suffix column
# Step2: Replacing the atom_0 column with atom_0_dummies
# Step3: Deleteing atom1 column and adding atom_1_dummies in place of it

train_df['type'] = type_suffix
train_df['atom_0'] = atom_0_dummies
train_df.drop('atom_1',axis=1,inplace=True)
train_df = pd.concat([train_df,atom_1_dummies],axis=1)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4659076 entries, 0 to 4659075
Data columns (total 40 columns):
id                          int64
molecule_name               object
atom_index_0                int64
atom_index_1                int64
type                        uint8
scalar_coupling_constant    float64
mulliken_charge_0           float64
mulliken_charge_1           float64
atom_0                      int64
x_0                         float64
y_0                         float64
z_0                         float64
x_1                         float64
y_1                         float64
z_1                         float64
potential_energy            float64
fc                          float64
sd                          float64
pso                         float64
dso                         float64
XXmag_0                     float64
YXmag_0                     float64
ZXmag_0                     float64
XYmag_0                     float64
YYmag_0                     float6

# Converting datatypes of the columns to compress the Dataset

In [24]:
type_dict = train_df.dtypes.to_dict()
type_dict

{'id': dtype('int64'),
 'molecule_name': dtype('O'),
 'atom_index_0': dtype('int64'),
 'atom_index_1': dtype('int64'),
 'type': dtype('uint8'),
 'scalar_coupling_constant': dtype('float64'),
 'mulliken_charge_0': dtype('float64'),
 'mulliken_charge_1': dtype('float64'),
 'atom_0': dtype('int64'),
 'x_0': dtype('float64'),
 'y_0': dtype('float64'),
 'z_0': dtype('float64'),
 'x_1': dtype('float64'),
 'y_1': dtype('float64'),
 'z_1': dtype('float64'),
 'potential_energy': dtype('float64'),
 'fc': dtype('float64'),
 'sd': dtype('float64'),
 'pso': dtype('float64'),
 'dso': dtype('float64'),
 'XXmag_0': dtype('float64'),
 'YXmag_0': dtype('float64'),
 'ZXmag_0': dtype('float64'),
 'XYmag_0': dtype('float64'),
 'YYmag_0': dtype('float64'),
 'ZYmag_0': dtype('float64'),
 'XZmag_0': dtype('float64'),
 'YZmag_0': dtype('float64'),
 'ZZmag_0': dtype('float64'),
 'XXmag_1': dtype('float64'),
 'YXmag_1': dtype('float64'),
 'ZXmag_1': dtype('float64'),
 'XYmag_1': dtype('float64'),
 'YYmag_1': dty

In [25]:
for i in type_dict.keys():
    typ = type_dict[i]
    if typ == np.float64:
        type_dict[i] = np.float32
    elif typ == np.int64:
        type_dict[i] = np.int32
    else:
        pass

In [26]:
type_dict

{'id': numpy.int32,
 'molecule_name': dtype('O'),
 'atom_index_0': numpy.int32,
 'atom_index_1': numpy.int32,
 'type': dtype('uint8'),
 'scalar_coupling_constant': numpy.float32,
 'mulliken_charge_0': numpy.float32,
 'mulliken_charge_1': numpy.float32,
 'atom_0': numpy.int32,
 'x_0': numpy.float32,
 'y_0': numpy.float32,
 'z_0': numpy.float32,
 'x_1': numpy.float32,
 'y_1': numpy.float32,
 'z_1': numpy.float32,
 'potential_energy': numpy.float32,
 'fc': numpy.float32,
 'sd': numpy.float32,
 'pso': numpy.float32,
 'dso': numpy.float32,
 'XXmag_0': numpy.float32,
 'YXmag_0': numpy.float32,
 'ZXmag_0': numpy.float32,
 'XYmag_0': numpy.float32,
 'YYmag_0': numpy.float32,
 'ZYmag_0': numpy.float32,
 'XZmag_0': numpy.float32,
 'YZmag_0': numpy.float32,
 'ZZmag_0': numpy.float32,
 'XXmag_1': numpy.float32,
 'YXmag_1': numpy.float32,
 'ZXmag_1': numpy.float32,
 'XYmag_1': numpy.float32,
 'YYmag_1': numpy.float32,
 'ZYmag_1': numpy.float32,
 'XZmag_1': numpy.float32,
 'YZmag_1': numpy.float32,


In [27]:
train_df = train_df.astype(type_dict)
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4659076 entries, 0 to 4659075
Data columns (total 40 columns):
id                          int32
molecule_name               object
atom_index_0                int32
atom_index_1                int32
type                        uint8
scalar_coupling_constant    float32
mulliken_charge_0           float32
mulliken_charge_1           float32
atom_0                      int32
x_0                         float32
y_0                         float32
z_0                         float32
x_1                         float32
y_1                         float32
z_1                         float32
potential_energy            float32
fc                          float32
sd                          float32
pso                         float32
dso                         float32
XXmag_0                     float32
YXmag_0                     float32
ZXmag_0                     float32
XYmag_0                     float32
YYmag_0                     float3

In [28]:
train_df.to_csv('dataset.csv')